In [56]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import losses

In [ ]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
dataset = tf.keras.utils.get_file("stack_overflow_16k.tar.gz", url, untar=True, cache_dir='.', cache_subdir='stackoverflow')

In [11]:
dataset_dir = os.path.join(os.path.dirname(dataset))
os.listdir(dataset_dir)

['test', 'stack_overflow_16k.tar.gz', 'README.md', 'train']

In [13]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

sample_file = os.path.join(train_dir, 'python/1755.txt')
with open(sample_file) as f:
    print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



In [16]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'stackoverflow/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [17]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print("Question", text_batch.numpy()[i])
        print("Label", label_batch.numpy()[i])

Question b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default const

In [18]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to csharp
Label 1 corresponds to java


In [19]:
## Create validation and test dataset
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'stackoverflow/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [20]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'stackoverflow/test',
    batch_size=batch_size
)

Found 8000 files belonging to 4 classes.


In [21]:
# Prepare the dataset for training
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

In [22]:
max_features = 5000
sequence_length = 500

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [23]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [24]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [25]:
# Retrieve a batch (of 32 questions and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized question", vectorize_text(first_question, first_label))

Question tf.Tensor(b'"set blank to quit on exception? i\'m using blank 3..i\'ve been looking around for an answer to this, but i haven\'t found it yet. basically, i\'m running several blank scripts into a game engine, and each script has its own entry point...i\'d rather not add try: except blocks through all of my code, so i was wondering if it\'s at all possible to tell blank to quit (or perhaps assign a custom function to that ""callback"") on finding its first error, regardless of where or what it found? ..currently, the game engine will continue after finding and hitting an error, making it more difficult than necessary to diagnose issues since running into one error may make a subsequent script not work (as it relies on variables that the error-ing script set, for example). any ideas? ..i know that i could redirect the console to a file to allow for easier scrolling, but just capturing the first error and stopping the game prematurely would be really useful...okay, a couple of ex

In [26]:
# vocabulary size
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

Vocabulary size: 5000


In [27]:
# vectorize the data
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [28]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [37]:
# Create the model
embedding_dim = 16

model = tf.keras.Sequential([
    layers.Embedding(max_features + 1, embedding_dim),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(4)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 16)          80016     
                                                                 
 dropout_4 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_2  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 68        
                                                                 
Total params: 80084 (312.83 KB)
Trainable params: 80084 (312.83 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [38]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [39]:
epochs = 50
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/50
200/200 [==============================] - 1s 3ms/step - loss: 1.3832 - accuracy: 0.2870 - val_loss: 1.3791 - val_accuracy: 0.4244
Epoch 2/50
200/200 [==============================] - 1s 3ms/step - loss: 1.3724 - accuracy: 0.3522 - val_loss: 1.3656 - val_accuracy: 0.4469
Epoch 3/50
200/200 [==============================] - 0s 2ms/step - loss: 1.3541 - accuracy: 0.4066 - val_loss: 1.3443 - val_accuracy: 0.4675
Epoch 4/50
200/200 [==============================] - 1s 3ms/step - loss: 1.3266 - accuracy: 0.4819 - val_loss: 1.3138 - val_accuracy: 0.5663
Epoch 5/50
200/200 [==============================] - 1s 3ms/step - loss: 1.2912 - accuracy: 0.5342 - val_loss: 1.2752 - val_accuracy: 0.6162
Epoch 6/50
200/200 [==============================] - 1s 3ms/step - loss: 1.2474 - accuracy: 0.5734 - val_loss: 1.2310 - val_accuracy: 0.6425
Epoch 7/50
200/200 [==============================] - 1s 3ms/step - loss: 1.2003 - accuracy: 0.6059 - val_loss: 1.1831 - val_accuracy: 0.6712
Epoch 

In [40]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 0s 745us/step - loss: 0.5507 - accuracy: 0.7955
Loss:  0.5507212281227112
Accuracy:  0.7954999804496765


In [41]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')

plt.title('Training and validation loss')
plt.xlabel('Epochs')

plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

In [45]:
# Export the model
export_model = tf.keras.Sequential([
    vectorize_layer,
    model,
    layers.Activation('sigmoid')
])

export_model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=False), optimizer='adam', metrics=['accuracy'])

loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

250/250 [==============================] - 0s 2ms/step - loss: 0.5507 - accuracy: 0.7955
0.7954999804496765


In [67]:
# Inference on new data
examples = [
    "debug public static void main(string[] args) {...}",
    "How do I extract keys from a dict into a list?",
    "unshift() not working on my array"
]

prediction = export_model.predict(examples)

1/1 [==============================] - 0s 19ms/step


In [68]:
languages = ['csharp', 'java', 'javascript', 'python']

for x in range(len(prediction)):
    print(examples[x])
    print(languages[np.argmax(prediction[x])])

debug public static void main(string[] args) {...}
java
How do I extract keys from a dict into a list?
python
unshift() not working on my array
javascript
